**Advanced AI Coursework**
*B716460*

Aims for this report:
 * Evaluate the comparative differences between TensorFlow and Pytorch.
 * Produce a comparative analysis from these given points:
   * Results from a range of different parameter sizes.
   * Results from a range of varying epochs for its training process.
   * Implementation of an older deep learning architecture, AlexNet.
   * Implementation of a more recent deep learning architecture, ResNeXt-50.
   * Produce results from the two architectures.

In [1]:
from tensorflow import keras
import numpy as np
import cv2
mnist = keras.datasets.mnist
cifar10 = keras.datasets.cifar10

(training_images, training_labels),(testing_images, testing_labels) = cifar10.load_data()
training_images, training_labels = training_images[0:10000], training_labels[0:10000]
testing_images, testing_labels = testing_images[0:1666], testing_labels[0:1666]

training_labels = keras.utils.to_categorical(training_labels)
testing_labels = keras.utils.to_categorical(testing_labels)

In [2]:
alexnet_image_resolution = (224, 224)

def resize_and_normalise_image_set(resolution: tuple, images: np.ndarray):
    print('Resizing %s from %s to %s' % ('images', images.shape[1:-1], alexnet_image_resolution))
    resized_images = np.empty((images.shape[0], 224, 224, 3), dtype=np.uint8)
    for (i, image) in enumerate(images):
        resized_images[i] = cv2.resize(image, dsize=resolution)
    # Normalise given images via division of max colour value.
    return resized_images
    # return images / 255.0

training_images = resize_and_normalise_image_set(alexnet_image_resolution, training_images)
print('Training images prepped.')
testing_images = resize_and_normalise_image_set(alexnet_image_resolution, testing_images)
print('Testing images prepped.')

Resizing images from (32, 32) to (224, 224)
Training images prepped.
Resizing images from (32, 32) to (224, 224)
Testing images prepped.


Create a Keras API sequential neural network model. This sequential model follows the AlexNet structure but has
been tweaked to conform with the CIFAR-10's dataset image size.

[AlexNet Source Code](https://medium.com/analytics-vidhya/alexnet-tensorflow-2-1-0-d398b7c76cf)

Original model was taken from this article and modified to adjust to the shape of the CIFAR-10 dataset images (32x32).

[![Illustrated model](images/alexnet.jpg)](https://www.mdpi.com/2072-4292/9/8/848/htm)

In [3]:

model = keras.models.Sequential([
    # 1st layer
  keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(224, 224, 3)),
  keras.layers.BatchNormalization(),
  keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
  keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  keras.layers.BatchNormalization(),
     # 3rd conv
  keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  keras.layers.BatchNormalization(),
    # 4th conv
  keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  keras.layers.BatchNormalization(),
    # 5th Conv
  keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  keras.layers.BatchNormalization(),
  keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  keras.layers.Flatten(),
  # To FC layer 1
  keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> keras.layers.Dropout(0.5),
  #To FC layer 2
  keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> keras.layers.Dropout(0.5),
  keras.layers.Dense(testing_labels.shape[1], activation='softmax')
])

In [4]:
model.compile(optimizer='sgd',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss. The Model.evaluate method checks the model's
performance against the "testing_image" dataset.

In [5]:
model.fit(training_images, training_labels, epochs=5)
model.summary()
model.evaluate(testing_images, testing_labels, verbose=2)

Epoch 1/5
313/313 [==============================] - 130s 360ms/step - loss: 2.5931 - accuracy: 0.3027

[1.4590142965316772, 0.558223307132721]